In [ ]:
from getpass import getpass
from owslib.csw import CatalogueServiceWeb as CSW
from pyproj import CRS, Transformer
from shapely.geometry import box, Polygon
from shapely.ops import transform

import pandas as pd
import geopandas as gpd
import logging

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
csw  = CSW("https://geoservicos.sgb.gov.br/geonetwork/srv/eng/csw", username="carlos.mota", password=getpass(prompt="Digite a senha"))

## Get results

In [ ]:
start = 1
page_size = 100

epsg_codes = {
    "WGS84": "EPSG:4326",
    "SIRGAS2000": "EPSG:4674",
    "SAD69": "EPSG:4291",
    "CórregoAlegre": "EPSG:4225",
}

results = []


while True:
    csw.getrecords2(startposition=start, maxrecords=page_size, esn="full")
    start = csw.results.get("nextrecord")
    logging.info(csw.results)

    for key, record in csw.records.items():
        selected_props = ("identifier", "title", "abstract", "subjects", "bbox")
        row = {prop: getattr(record, prop) for prop in selected_props}
        results.append(row)
    
    if not start:
        logging.info("Fetch results done")
        break

# Convert to DataFrame
metadata = (
    pd.DataFrame(results)
        .set_index("identifier")
)

# Sanitize bbox
bbox_df = (
    metadata[['bbox']].apply(
        lambda val: (float(val.bbox.minx), float(val.bbox.miny), float(val.bbox.maxx), float(val.bbox.maxy), val.bbox.crs.code) if val.bbox else (None,) * 5, 
        axis="columns", 
        result_type="expand"
    )
    .rename(columns={0: "minx", 1: "miny", 2: "maxx", 3: "maxy", 4: "crs"})
    .assign(
        crs = lambda df: (
            df.crs
                .str.replace("[\s\W]+", "", regex=True)     # Clear spaces and non-words
                .str.replace("1984", "84", regex=False)     # Normalize 1984 to 84 
                .str.replace("^4326$", "WGS84", regex=True) # Replace EPSG codes to WGS84
                .str.replace("^SouthAmericanDatum", "", regex=True) # Replace EPSG codes to WGS84
            .replace(epsg_codes)
            .fillna(epsg_codes['WGS84'])                    # set WGS84 as default CRS
        )
    )
    .convert_dtypes()
)

metadata = (
    metadata.drop("bbox", axis="columns")
        .join(bbox_df, validate="one_to_one")
)

metadata.to_parquet("metadata.parquet")
metadata.info()

## Sanitize BBOX

In [ ]:
def bbox_to_geometry(minx, miny, maxx, maxy, crs=None):
    try:
        bbox = box(minx, miny, maxx, maxy)
        
        if crs and crs != wgs84:            
            proj_src = pyproj.CRS(crs)
            proj_dst = pyproj.CRS(epsg_codes['WGS84'])

            project = Transformer.from_crs(proj_src, proj_dst, always_xy=True).transform
            bbox = transform(project, bbox)            
        
        return bbox
    
    except:
        return None

metadata_gdf = (
    gpd.GeoDataFrame(
        metadata.assign(
            geometry = lambda df: gpd.GeoSeries(df.apply(lambda df: bbox_to_geometry(df.minx, df.miny, df.maxx, df.maxy), axis="columns"))
        ),
        geometry="geometry"
    )
    .loc[
        lambda df: df.geometry.covered_by(box(-80, -40, -20, 20)) # Remove metadata not covered by Brazil bbpx
    ]
)

metadata_gdf.to_parquet("metadata-geo.parquet")

metadata_gdf.boundary.plot(edgecolor="black")